In [1]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
import pandas as pd
import numpy as np

In [3]:
import pandas as pd
data = pd.read_csv("HotelCustomersDataset.csv", delimiter=";")

In [4]:
data.sample()

,ID,Nationality,Age,DaysSinceCreation,NameHash,DocIDHash,AverageLeadTime,LodgingRevenue,OtherRevenue,BookingsCanceled,...,SRMediumFloor,SRBathtub,SRShower,SRCrib,SRKingSizeBed,SRTwinBed,SRNearElevator,SRAwayFromElevator,SRNoAlcoholInMiniBar,SRQuietRoom
48702,48703,FRA,53.0,291,0xDDB065924E1D2A6C761513044F6A645299B9D5156BB3...,0xD13938681415000B95E0C9F52C0F66CD384E3C657C5B...,0,0.0,0.0,0,...,0,0,0,0,1,0,0,0,0,0


In [5]:
data = data.drop(['NameHash','DocIDHash'], axis=1)

In [6]:
data.sample()

,ID,Nationality,Age,DaysSinceCreation,AverageLeadTime,LodgingRevenue,OtherRevenue,BookingsCanceled,BookingsNoShowed,BookingsCheckedIn,...,SRMediumFloor,SRBathtub,SRShower,SRCrib,SRKingSizeBed,SRTwinBed,SRNearElevator,SRAwayFromElevator,SRNoAlcoholInMiniBar,SRQuietRoom
12407,12408,PRT,74.0,865,161,128.7,45.0,0,0,1,...,0,0,0,0,0,1,0,0,0,0


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83590 entries, 0 to 83589
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ID                    83590 non-null  int64  
 1   Nationality           83590 non-null  object 
 2   Age                   79811 non-null  float64
 3   DaysSinceCreation     83590 non-null  int64  
 4   AverageLeadTime       83590 non-null  int64  
 5   LodgingRevenue        83590 non-null  float64
 6   OtherRevenue          83590 non-null  float64
 7   BookingsCanceled      83590 non-null  int64  
 8   BookingsNoShowed      83590 non-null  int64  
 9   BookingsCheckedIn     83590 non-null  int64  
 10  PersonsNights         83590 non-null  int64  
 11  RoomNights            83590 non-null  int64  
 12  DaysSinceLastStay     83590 non-null  int64  
 13  DaysSinceFirstStay    83590 non-null  int64  
 14  DistributionChannel   83590 non-null  object 
 15  MarketSegment      

### Computing RFM Scores for each customer

In [18]:
data['Recency'] = np.select(
    [
        data['DaysSinceLastStay'].between(1, 30),
        data['DaysSinceLastStay'].between(31, 50),
        data['DaysSinceLastStay'].between(51, 100),
        data['DaysSinceLastStay'].between(101, 400),
        data['DaysSinceLastStay'] > 400
    ],
    [
        5,
        4,
        3,
        2,
        1
    ],
    default=0
)

In [19]:
data['Frequency'] = np.select(
    [
        data['BookingsCheckedIn'] > 10,
        data['BookingsCheckedIn'].between(5, 10),
        data['BookingsCheckedIn'].between(2, 5),
        data['BookingsCheckedIn'] == 2,
        data['BookingsCheckedIn'] == 1,
    ],
    [
        5,
        4,
        3,
        2,
        1
    ],
    default=0
)

In [20]:
data["Total_revenues"] = data['LodgingRevenue'] + data['OtherRevenue']
data['Monetary'] = np.select(
    [
        data['Total_revenues'] >= 500,
        data['Total_revenues'].between(301, 499),
        data['Total_revenues'].between(201, 300),
        data['Total_revenues'].between(101, 200),
        data['Total_revenues'].between(1, 100),
        data['Total_revenues'] < 1
    ],
    [
        5,
        4,
        3,
        2,
        1,
        0
    ]
)
data = data.drop("Total_revenues", axis=1)

In [21]:
data.sample(5)

,ID,Nationality,Age,DaysSinceCreation,AverageLeadTime,LodgingRevenue,OtherRevenue,BookingsCanceled,BookingsNoShowed,BookingsCheckedIn,PersonsNights,RoomNights,DaysSinceLastStay,DaysSinceFirstStay,DistributionChannel,MarketSegment,Recency,Frequency,Monetary,RFM
66320,66321,BRA,30.0,151,0,0.0,0.0,0,0,0,0,0,-1,-1,Direct,Direct,0,0,0,0
13768,13769,GBR,69.0,844,99,375.3,30.0,0,0,1,6,3,847,847,Travel Agent/Operator,Other,1,1,4,6
71699,71700,DNK,56.0,111,148,835.0,150.5,0,0,1,10,5,116,116,Travel Agent/Operator,Other,2,1,5,8
17973,17974,FRA,57.0,784,86,234.0,44.5,0,0,1,6,3,787,787,Travel Agent/Operator,Other,1,1,3,5
78934,78935,PRT,41.0,57,0,0.0,0.0,0,0,0,0,0,-1,-1,Travel Agent/Operator,Other,0,0,0,0


In [15]:
data['RFM'] = data['Recency'] + data['Frequency'] + data['Monetary'] 

### Deleting room-specific columns

In [16]:
data = data.loc[:, ~data.columns.str.startswith('SR')]

In [17]:
data.sample(5)

,ID,Nationality,Age,DaysSinceCreation,AverageLeadTime,LodgingRevenue,OtherRevenue,BookingsCanceled,BookingsNoShowed,BookingsCheckedIn,PersonsNights,RoomNights,DaysSinceLastStay,DaysSinceFirstStay,DistributionChannel,MarketSegment,Recency,Frequency,Monetary,RFM
41580,41581,DEU,56.0,401,35,128.0,17.0,0,0,1,2,2,403,403,Travel Agent/Operator,Groups,1,1,2,4
73375,73376,AUS,1.0,100,0,0.0,0.0,0,0,0,0,0,-1,-1,Travel Agent/Operator,Other,0,0,0,0
22744,22745,GBR,44.0,678,252,158.4,168.5,0,0,1,4,2,680,680,Travel Agent/Operator,Other,1,1,4,6
65122,65123,USA,85.0,160,130,232.0,37.0,0,0,1,2,2,162,162,Direct,Direct,2,1,3,6
35180,35181,CHL,35.0,493,121,616.5,84.0,0,0,1,12,6,499,499,Direct,Direct,1,1,5,7


### Applying clustering algorithm

In [22]:
from sklearn.cluster import KMeans

# Set the number of clusters
num_clusters = 3

# Create a new dataframe with only the "RFM" column
rfm_data = data[['RFM']]

# Create a KMeans model and fit it to the data
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(rfm_data)

# Add the cluster labels to the original dataframe
data['Cluster'] = kmeans.labels_

In [30]:
data2 = data[["RFM", "Cluster"]]

In [31]:
data2.sample(5)

,RFM,Cluster
25722,6,1
33557,7,2
75859,11,2
3297,6,1
2329,0,0


In [32]:
data.to_csv('HotelRFM.csv', index=False)

### Showing the mean, minimum and maximum values in Recency, Frequency, Monetary and combined RFM for the three clusters

In [40]:
# Group the customers by their cluster labels and compute the mean RFM values for each cluster
cluster_means = data.groupby('Cluster')['Recency', 'Frequency', 'Monetary','RFM'].mean()
cluster_min = data.groupby('Cluster')['Recency', 'Frequency', 'Monetary','RFM'].min()
cluster_max = data.groupby('Cluster')['Recency', 'Frequency', 'Monetary','RFM'].max()


# Print the mean RFM values for each cluster
print(cluster_means)
print(cluster_min)
print(cluster_max)

          Recency  Frequency  Monetary       RFM
Cluster                                         
0        0.013959   0.013959  0.000000  0.027918
1        1.191377   1.005311  2.909623  5.106311
2        1.864928   1.096542  4.612058  7.573528
         Recency  Frequency  Monetary  RFM
Cluster                                   
0              0          0         0    0
1              0          1         0    3
2              1          1         0    7
         Recency  Frequency  Monetary  RFM
Cluster                                   
0              1          1         0    2
1              5          4         4    6
2              5          5         5   15


In [41]:
cluster_means

,Recency,Frequency,Monetary,RFM
Cluster,,,,
0,0.013959,0.013959,0.000000,0.027918
1,1.191377,1.005311,2.909623,5.106311
2,1.864928,1.096542,4.612058,7.573528


In [42]:
cluster_min

,Recency,Frequency,Monetary,RFM
Cluster,,,,
0,0,0,0,0
1,0,1,0,3
2,1,1,0,7


In [43]:
cluster_max

,Recency,Frequency,Monetary,RFM
Cluster,,,,
0,1,1,0,2
1,5,4,4,6
2,5,5,5,15


## Automation: every time a new customer is registered to our database...

In [44]:
import pandas as pd
data = pd.read_csv("HotelRFM.csv")

In [45]:
new_customer_id = [{
    "ID":84654,
    "Nationality":"IT",
    "Age":26,
    "DaysSinceCreation":23,
    "AverageLeadTime":44,
    "LodgingRevenue":400,
    "OtherRevenue":20,
    "BookingsCanceled":0,
    "BookingsNoShowed":0,
    "BookingsCheckedIn":1,
    "PersonsNights":2,
    "RoomNights":3,
    "DaysSinceLastStay":250,
    "DaysSinceFirstStay":253,
    "DistributionChannel":"Travel Agent/Operator",
    "MarketSegment":"Other",
    }
]

In [46]:
new_customer_id = pd.DataFrame.from_dict(new_customer_id, orient = "columns")

### Defining a function for the whole RFM creation

In [48]:
def RFM_creation(data):
    data['Recency'] = np.select(
    [
        data['DaysSinceLastStay'].between(1, 30),
        data['DaysSinceLastStay'].between(31, 50),
        data['DaysSinceLastStay'].between(51, 100),
        data['DaysSinceLastStay'].between(101, 400),
        data['DaysSinceLastStay'] > 400
    ],
    [
        5,
        4,
        3,
        2,
        1
    ],
    default=0) 
    data['Frequency'] = np.select(
    [
        data['BookingsCheckedIn'] > 10,
        data['BookingsCheckedIn'].between(5, 10),
        data['BookingsCheckedIn'].between(2, 5),
        data['BookingsCheckedIn'] == 2,
        data['BookingsCheckedIn'] == 1,
    ],
    [
        5,
        4,
        3,
        2,
        1
    ],
    default=0)
    data["Total_revenues"] = data['LodgingRevenue'] + data['OtherRevenue']
    data['Monetary'] = np.select(
        [
            data['Total_revenues'] >= 500,
            data['Total_revenues'].between(301, 499),
            data['Total_revenues'].between(201, 300),
            data['Total_revenues'].between(101, 200),
            data['Total_revenues'].between(1, 100),
            data['Total_revenues'] < 1
        ],
        [
            5,
            4,
            3,
            2,
            1,
            0
        ]
    )
    data['RFM'] = data['Recency'] + data['Frequency'] + data['Monetary']
    data = data[['Recency', 'Monetary', 'Frequency', 'RFM']]

In [50]:
RFM_creation(new_customer_id)

In [51]:
new_customer_id = new_customer_id.drop("Total_revenues", axis=1)

In [52]:
new_customer_id.sample()

,ID,Nationality,Age,DaysSinceCreation,AverageLeadTime,LodgingRevenue,OtherRevenue,BookingsCanceled,BookingsNoShowed,BookingsCheckedIn,PersonsNights,RoomNights,DaysSinceLastStay,DaysSinceFirstStay,DistributionChannel,MarketSegment,Recency,Frequency,Monetary,RFM
0,84654,IT,26,23,44,400,20,0,0,1,2,3,250,253,Travel Agent/Operator,Other,2,1,4,7


### Assigning the new customer to a pre-defined cluster

In [53]:
new_customer_id_pred = new_customer_id[["RFM"]]

In [54]:
new_customer_id_pred = kmeans.predict(new_customer_id_pred)
new_customer_id["Cluster"] = new_customer_id_pred[0]
new_customer_id.sample()

,ID,Nationality,Age,DaysSinceCreation,AverageLeadTime,LodgingRevenue,OtherRevenue,BookingsCanceled,BookingsNoShowed,BookingsCheckedIn,...,RoomNights,DaysSinceLastStay,DaysSinceFirstStay,DistributionChannel,MarketSegment,Recency,Frequency,Monetary,RFM,Cluster
0,84654,IT,26,23,44,400,20,0,0,1,...,3,250,253,Travel Agent/Operator,Other,2,1,4,7,2


### Calculating CLV for each customer with penalties

In [5]:
penalties_for_recency = {
    "5": 1,
    "4": 0.9,
    "3": 0.8,
    "2": 0.7,
    "1": 0.6,
    "0": 1
}

In [56]:
for index, row in data.iterrows():
    value_frequency = row["BookingsCheckedIn"]
    value_revenue = row["LodgingRevenue"] + row["OtherRevenue"]
    assigned_penalty = penalties_for_recency[row["Recency"]]
    data.at[index, "CLV"] = (value_frequency * value_revenue) * assigned_penalty

In [57]:
data.sample()

,ID,Nationality,Age,DaysSinceCreation,AverageLeadTime,LodgingRevenue,OtherRevenue,BookingsCanceled,BookingsNoShowed,BookingsCheckedIn,...,DaysSinceLastStay,DaysSinceFirstStay,DistributionChannel,MarketSegment,Recency,Frequency,Monetary,RFM,Cluster,CLV
47618,47619,FRA,30.0,302,0,0.0,0.0,0,0,0,...,-1,-1,Travel Agent/Operator,Other,0,0,0,0,0,0.0


In [58]:
cluster_clv = data.groupby("Cluster")["CLV"].mean()
print(cluster_clv)

Cluster
0      1.147241
1    159.703562
2    607.328488
Name: CLV, dtype: float64


In [62]:
cluster_clv = pd.DataFrame(cluster_clv)
cluster_clv

,CLV
Cluster,
0,1.147241
1,159.703562
2,607.328488


In [51]:
data["max_AC"] = data["CLV"]*0.3

In [52]:
data.sample()

,ID,Nationality,Age,DaysSinceCreation,AverageLeadTime,LodgingRevenue,OtherRevenue,BookingsCanceled,BookingsNoShowed,BookingsCheckedIn,...,DaysSinceFirstStay,DistributionChannel,MarketSegment,Recency,Frequency,Monetary,RFM,Cluster,CLV,max_AC
32452,32453,DEU,75.0,534,414,140.8,115.0,0,0,1,...,536,Travel Agent/Operator,Groups,1,1,3,5,1,153.48,46.044


### Creation of Dataframe for presentation

In [9]:
cc = pd.DataFrame({"Recency":[5,4,3,2,1,0],
                   "Penalty":[1,0.9,0.8,0.7,0.6,1]})

In [13]:
cc.set_index("Recency").T

Recency,5,4,3,2,1,0
Penalty,1.0,0.9,0.8,0.7,0.6,1.0
